This notebook exercises a bug in parameter by:
- Loading a data test
- Create a custom eval funciton (not sure if this is important or not)
- Running a a search with 100 permunations
- Waiting until the job completes by watching CPU usage
- Confirming the kernel is ok
- Running get_status and the kernal dies.

Viewing the logs its looks like all the tasks complete ok.

In [1]:
import graphlab as gl
gl.canvas.set_target('ipynb')
import tools 
import initialise

A newer version of GraphLab Create (v1.8.5) is available! Your current version is v1.8.4.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.
2016-03-18 16:04:23,804 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.8.4 started. Logging: /tmp/graphlab_server_1458277462.log


This non-commercial license of GraphLab Create is assigned to kevin.mcisaac@gmail.com and will expire on November 06, 2016. For commercial licensing options, visit https://dato.com/buy/.


## Automatic optimisation

In [2]:
train_data, test_data = initialise.load_data(reload_data=False)
folds = gl.cross_validation.KFold(train_data, 5)
train, validate=train_data.random_split(0.8, )

Loading saved processed data
306  features in total


In [3]:
def auc_eval(model, train, test): 
    return {'train_auc':    model.evaluate(train, metric='auc')['auc'],
            'validate_auc': model.evaluate(test,  metric='auc')['auc']}

In [11]:
params = {'target':'TARGET', 'random_seed':8923,
          'class_weights':'auto','metric':'auc',
          'early_stopping_rounds':10, 'max_iterations':300,
          'step_size': [0.1, 0.3],
          'max_depth': [4, 5, 6],
          'column_subsample': [0.8, 0.6, 0.5],
          'row_subsample': [1, 0.95]}

job = gl.grid_search.create((train, validate),gl.boosted_trees_classifier.create,
                             params, evaluator=auc_eval, perform_trial_run=False)
job.progress_bar()
result = job.get_results(wait=False).sort('validate_auc', ascending=False)
result

2016-03-18 16:13:06,385 [INFO] graphlab.deploy.job, 22: Validating job.
2016-03-18 16:13:06,392 [INFO] graphlab.deploy.map_job, 186: Validation complete. Job: 'Model-Parameter-Search-Mar-18-2016-16-13-0600000' ready for execution
2016-03-18 16:13:06,587 [INFO] graphlab.deploy.map_job, 192: Job: 'Model-Parameter-Search-Mar-18-2016-16-13-0600000' scheduled.
2016-03-18 16:13:06,683 [INFO] graphlab.deploy.job, 22: Validating job.
2016-03-18 16:13:06,692 [INFO] graphlab.deploy.map_job, 186: Validation complete. Job: 'Model-Parameter-Search-Mar-18-2016-16-13-0600001' ready for execution
2016-03-18 16:13:06,931 [INFO] graphlab.deploy.map_job, 192: Job: 'Model-Parameter-Search-Mar-18-2016-16-13-0600001' scheduled.
2016-03-18 16:13:07,083 [INFO] graphlab.deploy.job, 22: Validating job.
2016-03-18 16:13:07,094 [INFO] graphlab.deploy.map_job, 186: Validation complete. Job: 'Model-Parameter-Search-Mar-18-2016-16-13-0600002' ready for execution
2016-03-18 16:13:07,471 [INFO] graphlab.deploy.map_job

{'Canceled': 0,
 'Completed': 36,
 'Failed': 0,
 'Pending': 0,
 'Running': 0,
 'Total': 36}

Now get the job status. There will be a 10s+ pause then the error message "The kernel appears to have died. It will restart automatically."

In [12]:
result = job.get_results(wait=False).sort('validate_auc', ascending=False)
result

model_id,class_weights,column_subsample,early_stopping_rounds,max_depth,max_iterations,metric,random_seed
16,auto,0.6,10,5,300,auc,8923
27,auto,0.5,10,4,300,auc,8923
5,auto,0.8,10,6,300,auc,8923
17,auto,0.6,10,6,300,auc,8923
24,auto,0.5,10,4,300,auc,8923
10,auto,0.8,10,5,300,auc,8923
15,auto,0.6,10,4,300,auc,8923
13,auto,0.6,10,5,300,auc,8923
28,auto,0.5,10,5,300,auc,8923
4,auto,0.8,10,5,300,auc,8923


In [14]:
params = {'target':'TARGET', 'random_seed':8923,
          'class_weights':'auto','metric':'auc',
          'early_stopping_rounds':10, 'max_iterations':300,
          'step_size': [0.07],
          'max_depth': [4, 5, 6, 7],
          'column_subsample': [0.8, 0.6, 0.5],
          'row_subsample': [0.95]}

job = gl.grid_search.create((train, validate),gl.boosted_trees_classifier.create,
                             params, evaluator=auc_eval, perform_trial_run=False)
job.progress_bar()
result2 = job.get_results(wait=False).sort('validate_auc', ascending=False)
result2


2016-03-18 16:41:14,859 [INFO] graphlab.deploy.job, 22: Validating job.
2016-03-18 16:41:14,866 [INFO] graphlab.deploy.map_job, 186: Validation complete. Job: 'Model-Parameter-Search-Mar-18-2016-16-41-1400000' ready for execution
2016-03-18 16:41:15,139 [INFO] graphlab.deploy.map_job, 192: Job: 'Model-Parameter-Search-Mar-18-2016-16-41-1400000' scheduled.
2016-03-18 16:41:15,236 [INFO] graphlab.deploy.job, 22: Validating job.
2016-03-18 16:41:15,244 [INFO] graphlab.deploy.map_job, 186: Validation complete. Job: 'Model-Parameter-Search-Mar-18-2016-16-41-1400001' ready for execution
2016-03-18 16:41:15,568 [INFO] graphlab.deploy.map_job, 192: Job: 'Model-Parameter-Search-Mar-18-2016-16-41-1400001' scheduled.


model_id,class_weights,column_subsample,early_stopping_rounds,max_depth,max_iterations,metric,random_seed
2,auto,0.8,10,6,300,auc,8923
11,auto,0.5,10,7,300,auc,8923
5,auto,0.6,10,5,300,auc,8923
7,auto,0.6,10,7,300,auc,8923
10,auto,0.5,10,6,300,auc,8923
6,auto,0.6,10,6,300,auc,8923
8,auto,0.5,10,4,300,auc,8923
1,auto,0.8,10,5,300,auc,8923
3,auto,0.8,10,7,300,auc,8923
4,auto,0.6,10,4,300,auc,8923


In [15]:
xr = result.append(result2)

In [16]:
xr.sort('validate_auc', ascending=False)

model_id,class_weights,column_subsample,early_stopping_rounds,max_depth,max_iterations,metric,random_seed
16,auto,0.6,10,5,300,auc,8923
27,auto,0.5,10,4,300,auc,8923
5,auto,0.8,10,6,300,auc,8923
17,auto,0.6,10,6,300,auc,8923
24,auto,0.5,10,4,300,auc,8923
2,auto,0.8,10,6,300,auc,8923
10,auto,0.8,10,5,300,auc,8923
15,auto,0.6,10,4,300,auc,8923
13,auto,0.6,10,5,300,auc,8923
28,auto,0.5,10,5,300,auc,8923
